# CC Method

In [11]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time

#input variables here

#hot stream variables
Th1s = 250
Th1t = 40
Ch1 = 0.15

Th2s = 200 ##set to same as Th1s if only 1 hot stream
Th2t = 80 ##set to same as Th1t if only 1 hot stream
Ch2 = 0.25 #set to 0 if only 1 hot stream

#cold stream variables
Tc1s = 20
Tc1t = 180
Cc1 = 0.2

Tc2s = 140 ##set to same as Tc1s if only 1 cold stream
Tc2t = 230 ##set to same as Tc1t if only 1 cold stream
Cc2 = 0.3 ##set to 0 if only 1 cold stream

##temperature settings
Tmin = 10.00
Tref = min([Th1t,Th2t,Tc1s,Tc2s])

Using matplotlib backend: Qt5Agg


In [12]:
##initialising hot stream eqns
y1 = list(range(Th1t,Th1s + 1))
y2 = list(range(Th2t,Th2s + 1))

if Th1t > Th2t: 
    h1 = [Ch1*(item - Th1t) for item in y1]
    h2 = [Ch2*(item - Tref) for item in y2]
    
else:
    h1 = [Ch1*(item - Tref) for item in y1]
    h2 = [Ch2*(item - Th2t) for item in y2]    

## ych is temp range of composite hot stream, xh is enthalpy range of comp hot stream
ych = np.unique(np.concatenate((y1,y2)))

if max(y1) > max(y2):
    xh1 = np.interp(ych, y1, h1, left=0, right=0)
    xh2 = np.interp(ych, y2, h2, left=0, right=max(h2))
else: 
    xh1 = np.interp(ych, y1, h1, left=0, right=max(h1))
    xh2 = np.interp(ych, y2, h2, left=0, right=0)
    
xh = xh1 + xh2
plt.plot(xh, ych, label="Composite Hot Curve")

####initialising cold stream eqns
y3 = list(range(Tc1s, Tc1t + 1))
y4 = list(range(Tc2s, Tc2t + 1))

if Tc1s > Tc2s:
    c1 = [Cc1*(item - Tc1s) for item in y3]
    c2 = [Cc2*(item - Tref) for item in y4]
else:
    c1 = [Cc1*(item - Tref) for item in y3]
    c2 = [Cc2*(item - Tc2s) for item in y4]

## ycs is temp range of composite cold stream, xc is enthalpy range of comp cold stream

ycs = np.unique(np.concatenate((y3,y4)))

if max(y4) > max(y3): 
    xc1 = np.interp(ycs, y3, c1, left=0, right=max(c1))
    xc2 = np.interp(ycs, y4, c2, left=0, right=0)
else:
    xc1 = np.interp(ycs, y3, c1, left=0, right=0)
    xc2 = np.interp(ycs, y4, c2, left=0, right=max(c2))
    
xc = xc1+xc2
###
yca = (np.concatenate((ycs,[ycs[-1]] * (len(ych)-len(ycs)))))
xa = (np.concatenate((xc,[xc[-1]] * (len(ych)-len(ycs)))))
#plt.plot(xa, yca, label="Composite Cold Curve")

## solve for how much to shift cold curve along the x-axis(enthalpy)
ycinter = np.interp(xh, xc, ycs, left = 0, right = 0 )
tmin = min(ych-ycinter)

while tmin < Tmin:
    xc += 0.01
    ycinter = np.interp(xh, xc, ycs, left = 0, right = 0 )
    tmin = min(ych-ycinter)
    
tdiff = ych-ycinter
index = np.where(tdiff==tmin)

Tcoldpinch = ycinter[index][0]
Thotpinch = ych[index][0]
Qcmin = abs(xh[0] - xc[0])
Qhmin = abs(xh[-1] - xc[-1])

plt.plot(xc, ycs, label="Composite Cold Curve meeting Tmin")
plt.xlabel("Enthalpy")
plt.ylabel("Temperature")
plt.legend();

print(f"Tmin = {tmin}")
print(f"Qcmin = {Qcmin}")
print(f"Qhmin = {Qhmin}")
print(f"Hot Pinch Temp = {Thotpinch}")
print(f"Cold Pinch Temp = {Tcoldpinch}")

plt.show()

Tmin = 10.000000000001279
Qcmin = 9.999999999999767
Qhmin = 7.500000000003098
Hot Pinch Temp = 150
Cold Pinch Temp = 139.99999999999872
